In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization,Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from PIL import Image 
import numpy as np
import re
import os 
import glob 
import random
import logging
from tensorflow.keras import backend as K
import math
tf.get_logger().setLevel(logging.ERROR)

In [2]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same", use_bias=False)(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape, n_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    c1, p1 = encoder_block(inputs, 16)
    c2, p2 = encoder_block(p1, 32)
    c3, p3 = encoder_block(p2, 64)
    c4, p4 = encoder_block(p3, 128)

    c5 = conv_block(p4, 256) #Bridge
    c6 = decoder_block(c5, c4, 128)
    c7 = decoder_block(c6, c3, 64)
    c8 = decoder_block(c7, c2, 32)
    c9 = decoder_block(c8, c1, 16)

    if n_classes == 1:  #Binary
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    
    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(c9)
    model = Model(inputs, outputs, name="U-Net")
    return model

In [3]:
class DataReader(object):
    def __init__(self, img_dir, mask_dir, is_training=True, img_size=(128, 128)):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.imgs_path = glob.glob(f"{self.img_dir}/*.jpg")
        self.num_imgs = len(self.imgs_path)
        self.img_h, self.img_w = img_size
        self.is_training = is_training
        
    
    def __len__(self):
        return self.num_imgs
    
    
    def find_mask(self, img_path):
        mask_path = re.sub(self.img_dir, self.mask_dir, img_path)
        mask_path = re.sub(".jpg", ".png", mask_path)
        return mask_path if os.path.exists(mask_path) else None
    
    
    def load_normalized_resize_img(self, img_path, is_mask=False, resize_method=tf.image.ResizeMethod.BILINEAR):
        img = tf.io.read_file(img_path)
        channels = 1 if is_mask else 3
        # tf.image.decode_image if passing dtype=tf.float32, the value is range from [0,1] no need to normalized again 
        img = tf.image.decode_image(img, channels=channels, dtype=tf.float32)
        img = tf.image.resize_with_pad(img, self.img_h, self.img_w, method=resize_method)
        return img
        
    
    def __getitem__(self, index):
        img_path = self.imgs_path[index]
        mask_path = self.find_mask(img_path)
        
        img = self.load_normalized_resize_img(img_path)
        if mask_path:
            mask = self.load_normalized_resize_img(mask_path, is_mask=True)
        else:
            mask = tf.zeros((self.img_h, self.img_w, 1), dtype=tf.float32)
        mask = tf.where(mask > 0, 1., 0)
        
        if self.is_training:
            img, mask = self.geo_transform(img, mask)
            img = self.color_transform(img)
            img = tf.clip_by_value(img, 0, 1)
        
        return img, mask 
    
    
    def iter(self):
        for i in range(self.num_imgs):
            yield self[i]
            
    
    def geo_transform(self, img, mask):
        random_number = random.random()
        if random_number < 0.2:
            img = tf.image.flip_left_right(img)
            mask = tf.image.flip_left_right(mask)
        elif random_number < 0.4:
            img = tf.image.flip_up_down(img)
            mask = tf.image.flip_up_down(mask)
        elif random_number < 0.6:
            img = tf.image.rot90(img, 1)
            mask = tf.image.rot90(mask, 1)
        elif random_number < 0.8:
            img = tf.image.rot90(img, 3)
            mask = tf.image.rot90(mask, 3)
        return img, mask 
    
    
    def color_transform(self, img):
        random_number = random.random()
        if random_number < 0.2:
            img = tf.image.adjust_gamma(img, 0.6)
        elif random_number < 0.6:
            img = tf.image.random_brightness(img, max_delta=0.1)
        return img

In [4]:
"""
Loss function and metrics
"""
def jaccard_coefficient(y_true, y_pred, smooth=1.):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection
    return (intersection + smooth) / (union + smooth)


def jaccard_loss(y_true, y_pred):
    return 1 - jaccard_coefficient(y_true, y_pred)


def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coefficient(y_true, y_pred)
    return loss

# Trainning

In [48]:
strategy = tf.distribute.MirroredStrategy()
NUM_REPLICAS = strategy.num_replicas_in_sync
BATCH_PER_REPLICAS = 32
GLOBAL_BATCH = BATCH_PER_REPLICAS * NUM_REPLICAS
AUTOTUNE = tf.data.experimental.AUTOTUNE

img_dir = "/app/seg/data/seg_training/images"
label_dir = "/app/seg/data/seg_training/labels"
data_reader = DataReader(img_dir, label_dir, is_training=True, img_size=(128, 128))
dataset = tf.data.Dataset.from_generator(data_reader.iter,
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=([128, 128, 3], [128, 128, 1]))
dataset = dataset.batch(GLOBAL_BATCH, drop_remainder=True).prefetch(AUTOTUNE)
# dataset = strategy.experimental_distribute_dataset(dataset)

num_examples = len(data_reader)
steps_per_epoch = num_examples // GLOBAL_BATCH

In [49]:
with strategy.scope():
    model = build_unet(input_shape=(128,128,3), n_classes=1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss=jaccard_loss, metrics=[jaccard_coefficient])
#, steps_per_epoch=steps_per_epoch
history = model.fit(dataset, epochs=50)

Epoch 1/50
59/59 [==============================] - 54s 908ms/step - loss: 0.4703 - jaccard_coefficient: 0.5297
Epoch 2/50
59/59 [==============================] - 53s 907ms/step - loss: 0.3338 - jaccard_coefficient: 0.6662
Epoch 3/50
59/59 [==============================] - 54s 909ms/step - loss: 0.2660 - jaccard_coefficient: 0.7340
Epoch 4/50
59/59 [==============================] - 52s 885ms/step - loss: 0.2295 - jaccard_coefficient: 0.7705
Epoch 5/50
59/59 [==============================] - 53s 900ms/step - loss: 0.2112 - jaccard_coefficient: 0.7888
Epoch 6/50
59/59 [==============================] - 54s 907ms/step - loss: 0.2004 - jaccard_coefficient: 0.7996
Epoch 7/50
59/59 [==============================] - 53s 904ms/step - loss: 0.1921 - jaccard_coefficient: 0.8079
Epoch 8/50
59/59 [==============================] - 53s 891ms/step - loss: 0.1870 - jaccard_coefficient: 0.8130
Epoch 9/50
59/59 [==============================] - 52s 888ms/step - loss: 0.1805 - jaccard_coefficient:

In [ ]:
model.save("jaccard_coefficient_mode.h5")

# Export Model

In [14]:
checkpoint_path = "/app/seg/jaccard_coefficient_mode.h5"
model = build_unet(input_shape=(128,128,3), n_classes=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss=jaccard_loss, metrics=[jaccard_coefficient])
model.load_weights(checkpoint_path)

In [19]:
def representative_dataset(dataset_dir):
    files = glob.glob(dataset_dir + "/*.jpg")
    print(len(files))
    for file in files:
        image = Image.open(file)
        image = image.resize((128, 128))
        image = np.array(image) / 255.
        yield [image]

In [20]:
dataset_dir = "/app/seg/data/images/train"
rep = representative_dataset(dataset_dir)

In [23]:
model.input.set_shape((1,) + model.input.shape[1:])
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(1, 128, 128, 3)]   0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 128, 128, 16) 432         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_90 (BatchNo (None, 128, 128, 16) 64          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 128, 128, 16) 0           batch_normalization_90[0][0]     
______________________________________________________________________________________________

In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# save_path = "/app/seg/jac_model.tflite"

# with open(save_path, 'wb') as f:
#     f.write(tflite_model)

TypeError: 'generator' object is not callable